### Example code for testing the implementation & comparing with classical GI algorithm
* Note: For testing all pairs (for families with more than 2 graphs), use `parallel.py` that supports command line arguments. Run `python parallel.py -h` to get an overview. **Running the functions serially is strongly discouraged.**
* In each case below, `True` implies isomorphism, which would be a false-positive since the graphs are non-isomorphic, and vice-versa. $p = 1$ and $p = 2$ CTQWs produce false-positives, while the classical test and $p = 3$ & $p = 4$ CTQWs do not.

In [1]:
# Local import
from utils import *

In [2]:
# Read in graph family
family_path = "16-6-2-2.txt"
data = DATA / family_path
graphs = read_data(data)
graphs.shape # (N(graphs), N(vertices), N(vertices)) # 16-6-2-2 has only 2 graphs.

(2, 16, 16)

#### Testing using classical algorithm

In [3]:
GI_classical_test(graphs[0], graphs[1])

False

#### Testing using quantum algorithm with specific CTQW parameters

In [4]:
# 1-particle Bosonic CTQW
GI_quantum_test(graphs[0], graphs[1], p=1, ptype="bos")

True

In [5]:
# 2-particle Fermionic CTQW
GI_quantum_test(graphs[0], graphs[1], p=2, ptype="ferm")

True

In [6]:
# 3-particle Fermionic CTQW
GI_quantum_test(graphs[0], graphs[1], p=3, ptype="ferm", tol=1e-11)

False

In [7]:
# 4-particle Bosonic CTQW
# GI_quantum_test(graphs[0], graphs[1], p=4, ptype="bos", tol=1e-10) # WARNING: MEMORY INTENSIVE (~156 GB)

False


### To compare with analytical expression for $U_{p = 1}$

* The following code tests whether the output from the code here matches the analytical expression for $|U_{p = 1}|$ from Gamble et al.
* It also confirms that 1-particle walks are insufficient to distinguish between isomorphic graphs.

In [8]:
import numpy as np, scipy as sc

ex = sc.linalg.expm
c, s, sq = np.cos, np.sin, np.sqrt

t = 1. # time
sq2 = sq(2)
sq2t = sq2 * t
cs = c(sq2t)
ss = s(sq2t)

In [9]:
## Analytical expressions from Gamble et al - Eqns. 28 & 29
U_A_paper = np.array([
    [cs / 2 + 0.5, -1j * ss / sq2, cs / 2 - 0.5],
    [-1j * ss / sq2, cs, -1j * ss / sq2],
    [cs / 2 - 0.5, -1j * ss / sq2, cs / 2 + 0.5]
])
U_B_paper = np.array([
    [cs, -1j * ss / sq2, 1j * ss / sq2],
    [-1j * ss / sq2, cs / 2 + 0.5, -cs / 2 + 0.5,],
    [1j * ss / sq2, -cs / 2 + 0.5, cs / 2 + 0.5]
])

## Own calc
# Adjacency matrix from Gamble et al - Eqn. 26
A = np.array([
    [1., 1., 0.],
    [1., 1., 1.],
    [0., 1., 1.]
])

# Hamiltonians
H_A = -A

# Evolution matrix
U_A_own = ex(-1j * H_A * t)

In [10]:
# Confirming that the analytical expressions from the paper gives the same result as my code
np.allclose(np.abs(U_A_own), np.abs(U_A_paper))

True

In [11]:
# Confirming that a 1-particle walk is insufficent to distinguish the graphs
np.allclose(np.sort(np.abs(U_A_paper).flatten()), np.sort(np.abs(U_B_paper).flatten())) # `True` denotes a false-positive. 

True